# Agenda

1. Chat Completions
   * Message Types
   * Request Parameters
   * Prompt Engineering
   * Structured Output
   * Tool Calling
2. Retrieval Augmented Generation
   * Vector Databases
   * Visualizing Semantic Similarity
   * Hybrid Search and Reranking
3. Promptflow
   * Anatomy of a flow
   * Evaluations + Benchmarking


# Chat Completions

## Message Types

In [ ]:
print("hello world!")

## Request Parameters

In [ ]:
print("hello world!")

## Prompt Engineering

In [ ]:
print("hello world!")

## Structured Output

In [ ]:
print("hello world!")

## Tool Calling
* Normal Tools with OpenAPI specs
* Semantic Kernel Auto Tool Calling

In [ ]:
print("hello world!")

# Retrieval Augmented Generation

## Vector Databases

In [ ]:
print("hello world!")

## Visualizing Semantic Similarity

In [ ]:
print("hello world!")

## Hybrid Search and Reranking

In [ ]:
print("hello world!")

# Promptflow

## Anatomy of a flow

In [1]:
print("hello world")

hello world


## Evaluations + Benchmarking

In [1]:
print("hello world")

hello world
